<a href="https://colab.research.google.com/github/sara-yasmin14/ASD-Detection-Using-AI-ML/blob/main/HHO_(2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import cv2
import os
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from google.colab import drive
from sklearn.preprocessing import LabelEncoder
import openpyxl

# Mount Google Drive
drive.mount('/content/drive')

# Specify the directory containing images on Google Drive
image_directory = '/content/drive/My Drive/NORMAL/'

Mounted at /content/drive


In [ ]:
# Function to load images and extract features
def load_images_and_extract_features(directory):
    images = []
    labels = []
    for filename in os.listdir(directory):
        if filename.endswith(".jpg") or filename.endswith(".png"):
            img_path = os.path.join(directory, filename)
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)  # Read image as grayscale
            img = cv2.resize(img, (128, 128))  # Resize images to a fixed size
            img_flat = img.flatten()  # Flatten the image to create feature vector
            images.append(img_flat)
            # Extract label from filename, assuming format: 'label_imagename.jpg'
            label = filename.split('_')[0]
            labels.append(label)
    return np.array(images), np.array(labels)

# Load images and extract features
data, labels = load_images_and_extract_features(image_directory)

# Encode labels to integers if they are not already
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)

In [ ]:
def feature_selection_objective(selected_features, data, labels):
    selected_indices = np.where(selected_features == 1)[0]
    if len(selected_indices) == 0:
        return 0
    X_selected = data[:, selected_indices]
    X_train, X_test, y_train, y_test = train_test_split(X_selected, labels, test_size=0.3, random_state=42)
    model = RandomForestClassifier()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    return accuracy_score(y_test, y_pred)

In [ ]:
# Harris Hawk Optimization for feature selection
def hho_for_feature_selection(num_hawks, num_features, max_iter, data, labels):
    hawks = np.random.randint(0, 2, (num_hawks, num_features))
    fitness = np.zeros(num_hawks)

    for i in range(num_hawks):
        fitness[i] = feature_selection_objective(hawks[i], data, labels)

    best_hawk = hawks[np.argmax(fitness)]
    best_fitness = np.max(fitness)

    for iter in range(max_iter):
        print(f"Iteration {iter + 1}/{max_iter}")
        for i in range(num_hawks):
            E1 = 2 * (1 - (iter / max_iter))
            E0 = 2 * np.random.rand() - 1
            E = E1 * E0

            if np.abs(E) >= 1:
                q = np.random.rand()
                if q >= 0.5:
                    hawks[i] = hawks[np.random.randint(num_hawks)] - np.random.rand() * np.abs(hawks[np.random.randint(num_hawks)] - 2 * np.random.rand() * hawks[i])
                else:
                    hawks[i] = (best_hawk - hawks.mean(axis=0)) - np.random.rand() * ((np.random.rand() * 20 - 10) * np.ones(num_features))
            else:
                r = np.random.rand()
                if r >= 0.5 and np.abs(E) < 0.5:
                    hawks[i] = best_hawk - E * np.abs(best_hawk - hawks[i])
                elif r >= 0.5 and np.abs(E) >= 0.5:
                    hawks[i] = best_hawk - E * np.abs(best_hawk - hawks.mean(axis=0))
                elif r < 0.5 and np.abs(E) < 0.5:
                    hawks[i] = best_hawk - E * np.abs(best_hawk - 2 * np.random.rand() * hawks[i])
                elif r < 0.5 and np.abs(E) >= 0.5:
                    hawks[i] = best_hawk - E * np.abs(best_hawk - hawks.mean(axis=0))

        for i in range(num_hawks):
            fitness[i] = feature_selection_objective(hawks[i], data, labels)

        current_best = hawks[np.argmax(fitness)]
        current_best_fitness = np.max(fitness)

        if current_best_fitness > best_fitness:
            best_hawk = current_best
            best_fitness = current_best_fitness

        print(f"Best Fitness so far: {best_fitness}")

    return best_hawk, best_fitness

# Example usage
num_hawks = 10
num_features = data.shape[1]
max_iter = 50

best_features, best_fitness = hho_for_feature_selection(num_hawks, num_features, max_iter, data, labels)
print("Best Features:", best_features)
print("Best Fitness:", best_fitness)

# Prepare data for Excel
feature_names = [f'feature_{i}' for i in range(num_features)]
best_feature_names = [feature_names[i] for i in range(len(best_features)) if best_features[i] == 1]
print("Selected Features:", best_feature_names)  # Debugging statement

# Export results to Excel
results_df = pd.DataFrame({'Selected Features': best_feature_names, 'Fitness': [best_fitness] * len(best_feature_names)})
print(results_df)  # Debugging statement
results_df.to_excel('/content/drive/My Drive/hho_feature_selection_results.xlsx', index=False)
print("Results saved to Excel.")

Iteration 1/50
Best Fitness so far: 0.0
Iteration 2/50
Best Fitness so far: 0.0
Iteration 3/50
Best Fitness so far: 0.0
Iteration 4/50
Best Fitness so far: 0.0
Iteration 5/50
Best Fitness so far: 0.0
Iteration 6/50
Best Fitness so far: 0.0
Iteration 7/50
Best Fitness so far: 0.0
Iteration 8/50
Best Fitness so far: 0.0
Iteration 9/50
Best Fitness so far: 0.0
Iteration 10/50
Best Fitness so far: 0.0
Iteration 11/50
Best Fitness so far: 0.0
Iteration 12/50
Best Fitness so far: 0.0
Iteration 13/50
Best Fitness so far: 0.0
Iteration 14/50
Best Fitness so far: 0.0
Iteration 15/50
Best Fitness so far: 0.0
Iteration 16/50
Best Fitness so far: 0.0
Iteration 17/50
Best Fitness so far: 0.0
Iteration 18/50
Best Fitness so far: 0.0
Iteration 19/50
Best Fitness so far: 0.0
Iteration 20/50
Best Fitness so far: 0.0
Iteration 21/50
Best Fitness so far: 0.0
Iteration 22/50
Best Fitness so far: 0.0
Iteration 23/50
Best Fitness so far: 0.0
Iteration 24/50
Best Fitness so far: 0.0
Iteration 25/50
Best Fitn